In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [3]:
import kagglehub

# Download the dataset to a local directory
path = kagglehub.dataset_download("salahuddinahmedshuvo/grocery-inventory-and-sales-dataset")

print("Path to dataset files:", path)

c:\Users\Nikunj\miniconda3\envs\tf_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\Nikunj\.cache\kagglehub\datasets\salahuddinahmedshuvo\grocery-inventory-and-sales-dataset\versions\1


In [4]:
import os

path = r"C:\Users\Nikunj\.cache\kagglehub\datasets\salahuddinahmedshuvo\grocery-inventory-and-sales-dataset\versions\1"
print(os.listdir(path))

['Grocery_Inventory_and_Sales_Dataset.csv']


In [5]:
import pandas as pd

file_path = r"C:\Users\Nikunj\.cache\kagglehub\datasets\salahuddinahmedshuvo\grocery-inventory-and-sales-dataset\versions\1\Grocery_Inventory_and_Sales_Dataset.csv"
df = pd.read_csv(file_path)

print(df.head())
print(df.columns)

    Product_ID     Product_Name             Catagory  Supplier_ID  \
0  29-205-1132       Sushi Rice      Grains & Pulses  38-037-1699   
1  40-681-9981   Arabica Coffee            Beverages  54-470-2479   
2  06-955-3428       Black Rice      Grains & Pulses  54-031-2945   
3  71-594-6552  Long Grain Rice      Grains & Pulses  63-492-7603   
4  57-437-1828             Plum  Fruits & Vegetables  54-226-4308   

  Supplier_Name  Stock_Quantity  Reorder_Level  Reorder_Quantity Unit_Price  \
0     Jaxnation              22             72                70     $4.50    
1       Feedmix              45             77                 2    $20.00    
2        Vinder              30             38                83     $6.00    
3    Brightbean              12             59                62     $1.50    
4    Topicstorm              37             30                74     $4.00    

  Date_Received Last_Order_Date Expiration_Date      Warehouse_Location  \
0     8/16/2024       6/29/2024    

In [6]:
data = pd.DataFrame(df)
data.head()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price,Date_Received,Last_Order_Date,Expiration_Date,Warehouse_Location,Sales_Volume,Inventory_Turnover_Rate,Status
0,29-205-1132,Sushi Rice,Grains & Pulses,38-037-1699,Jaxnation,22,72,70,$4.50,8/16/2024,6/29/2024,9/19/2024,48 Del Sol Trail,32,19,Discontinued
1,40-681-9981,Arabica Coffee,Beverages,54-470-2479,Feedmix,45,77,2,$20.00,11/1/2024,5/29/2024,5/8/2024,36 3rd Place,85,1,Discontinued
2,06-955-3428,Black Rice,Grains & Pulses,54-031-2945,Vinder,30,38,83,$6.00,8/3/2024,6/10/2024,9/22/2024,3296 Walton Court,31,34,Backordered
3,71-594-6552,Long Grain Rice,Grains & Pulses,63-492-7603,Brightbean,12,59,62,$1.50,12/8/2024,2/19/2025,4/17/2024,3 Westerfield Crossing,95,99,Active
4,57-437-1828,Plum,Fruits & Vegetables,54-226-4308,Topicstorm,37,30,74,$4.00,7/3/2024,10/11/2024,10/5/2024,15068 Scoville Court,62,25,Backordered


In [7]:
data['Unit_Price'] = data['Unit_Price'].replace('[\$,]', '', regex=True).astype(float)

# Drop rows with missing target
data.dropna(subset=['Sales_Volume'], inplace=True)

# Features
numerical = ['Stock_Quantity', 'Reorder_Level', 'Unit_Price', 'Inventory_Turnover_Rate']
categorical = ['Catagory', 'Warehouse_Location', 'Supplier_Name']

# One-hot encode categoricals
data_encoded = pd.get_dummies(data[categorical], drop_first=True)

# Final feature matrix and target
X = pd.concat([data[numerical], data_encoded], axis=1)
y = data['Sales_Volume']

# Scale numerical features
scaler = StandardScaler()
X[numerical] = scaler.fit_transform(X[numerical])

In [8]:
def build_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mae_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(X)):
    print(f"\n Fold {fold+1}")

    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model = build_model(X.shape[1])

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )


 Fold 1

 Fold 2

 Fold 3

 Fold 4


c:\Users\Nikunj\miniconda3\envs\tf_env\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



 Fold 5


In [10]:
history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        callbacks=[early_stop],
        verbose=0
    )

In [11]:
val_mae = model.evaluate(X_val, y_val, verbose=0)[1]
print(f"✅ Fold {fold+1} Validation MAE: {val_mae:.2f}")
mae_scores.append(val_mae)

✅ Fold 5 Validation MAE: 20.03


In [12]:
print("\n📊 Average Validation MAE across folds:", np.mean(mae_scores))


📊 Average Validation MAE across folds: 20.026180267333984
